# Cluster differential expressed genes

In [1]:
library_load <- suppressMessages(
    
    list(
        
        # Seurat 
        library(Seurat), 
        
        # Data 
        library(tidyverse), 
        library(openxlsx)
        
    )
)

In [2]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [3]:
# Source files
source("plotting_global.R")
source("bin/SeuratDEG.R")

In [4]:
options(warn=-1)

# Parameter settings

In [12]:
so_file_nacl <- "data/object/seurat_sct_nacl.rds"
so_file_cpg <- "data/object/seurat_sct_cpg.rds"

# Import Seurat 

In [13]:
so_nacl <- readRDS(so_file_nacl)
so_cpg <- readRDS(so_file_cpg)

so <- list(so_nacl, so_cpg)

In [15]:
names(so) <- c("NaCl", "CpG")

# Cluster annotation 

In [16]:
ref <- readRDS("data/haemosphere/se_haemosphere.rds")

In [ ]:
source("bin/SingleRQC.R")
so <- lapply(so, SingleRSeurat, ref=ref, cluster="SCT_snn_res.0.8")

In [18]:
set_factor_level <- function(so) {
    
    so$main_labels_cluster <- factor(so$main_labels_cluster, factor(names(color$main_labels_haemosphere)))
    so$fine_labels_cluster <- factor(so$fine_labels_cluster, factor(names(color$fine_labels_haemosphere)))
    
    return(so)
    
}

so <- lapply(so, set_factor_level)

# Compute cluster differential expressed genes 

In [ ]:
cluster_deg <- lapply(so, FindAllMarkers)

In [22]:
cluster_deg_nacl <- cluster_deg[["NaCl"]]
so_nacl <- so[["NaCl"]]

cluster_deg_cpg <- cluster_deg[["CpG"]]
so_cpg <- so[["CpG"]]

In [23]:
cluster_deg_nacl <- dplyr::left_join(cluster_deg_nacl, unique(so_nacl@meta.data[, c("SCT_snn_res.0.8", "main_labels_cluster", "fine_labels_cluster")]), by=c("cluster"="SCT_snn_res.0.8"))

cluster_deg_cpg <- dplyr::left_join(cluster_deg_cpg, unique(so_cpg@meta.data[, c("SCT_snn_res.0.8", "main_labels_cluster", "fine_labels_cluster")]), by=c("cluster"="SCT_snn_res.0.8"))

In [24]:
cluster_deg_nacl <- split(cluster_deg_nacl, f=cluster_deg_nacl$cluster)
cluster_deg_nacl <- cluster_deg_nacl[as.character(0:(length(cluster_deg_nacl)-1))]
names(cluster_deg_nacl) <- paste0("cluster_", names(cluster_deg_nacl))

cluster_deg_cpg <- split(cluster_deg_cpg, f=cluster_deg_cpg$cluster)
cluster_deg_cpg <- cluster_deg_cpg[as.character(0:(length(cluster_deg_cpg)-1))]
names(cluster_deg_cpg) <- paste0("cluster_", names(cluster_deg_cpg))

In [25]:
write.xlsx(cluster_deg_nacl, file="result/cluster_deg/cluster_deg_nacl.xlsx", overwrite=TRUE)

write.xlsx(cluster_deg_cpg, file="result/cluster_deg/cluster_deg_cpg.xlsx", overwrite=TRUE)